### NAMA : ADI HERIANTO RAJAGUKGUK
### BATCH : 22
### INGIN MENJADI DATA SCIENTIST DI DUNIA PERBANKAN

# A/B TEST

Kasus yang akan digunakan adalah membuka akun deposit untuk
nasabah bank. Pada kesempatan kali ini data scientist diminta tolong
untuk menganalisa hasil ab-testing suatu perusahaan bank. Apakah
jumlah campaign mampu meningkatkan keinginan user untuk
membuka akun deposit atau tidak?

## LOAD DATASET

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st

#### FROM DRIVE

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
import pandas as pd
bank = pd.read_csv('/content/gdrive/My Drive/PORTFOLIO STATISTIK/bank-full.csv', sep = ';')
bank

#### FROM DIRECTORY (INTERNAL PC)

In [ ]:
bank = pd.read_csv('bank-full.csv', sep = ';')
bank.sample(5)

In [ ]:
bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [ ]:
bank.shape

(45211, 17)

### 1 Pemahaman konsep A/B Testing

1. Apa nama experimentnya?
- AB TEST pengaruh jumlah campaign terhadap pembukaan akun deposit bank oleh user.
2. Definisikan Hipotesis
- H0: Pengadaan campaign mempengaruhi jumlah pembukaan akun deposit oleh user.
- H1: Pengadaan campaign tidak mempengaruhi jumlah pembukaan akun deposit oleh user.
3. Siapa participant-nya?
- User/Customer/nasabah yang ditargetkan dalam campaign Bank
4. Variabel yang akan diuji apa?
- Jumlah Campaign
5. Metrics apa yang akan digunakan?
- Metrics yang sangat bersentuhan dengan hipotesis, yaitu user membuka akun atau tidak.
(bisa menggunakan variabel y, Yes rate/conversion)

## EDA

In [ ]:
bank.describe()

,age,balance,day,duration,campaign,pdays,previous
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,40.936210,1362.272058,15.806419,258.163080,2.763841,40.197828,0.580323
std,10.618762,3044.765829,8.322476,257.527812,3.098021,100.128746,2.303441
min,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,33.000000,72.000000,8.000000,103.000000,1.000000,-1.000000,0.000000
50%,39.000000,448.000000,16.000000,180.000000,2.000000,-1.000000,0.000000
75%,48.000000,1428.000000,21.000000,319.000000,3.000000,-1.000000,0.000000
max,95.000000,102127.000000,31.000000,4918.000000,63.000000,871.000000,275.000000


In [ ]:
bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [ ]:
print('Ada sebanyak', bank[bank['y']=='yes'].shape[0], 'customer, atau', round(bank[bank['y']=='yes'].shape[0]/bank.shape[0]*100), '% yang membuka akun deposito')

Ada sebanyak 5289 customer, atau 12 % yang membuka akun deposito


### Sample Size using Slovin

In [ ]:
#diketahui :
N = 45211
e = 0.05

In [ ]:
slovin = N/(1+N * e**2)
slovin 

396.4920742803271

#### sample observation = 397

In [ ]:
samples = data_sampel1 = bank.sample(n=397, random_state=0).reset_index()

6. Berapa sample size dan durasi experiment?
- sample size : 397
- Durasi eksperimen : Hingga seluruh sample selesai diuji

In [ ]:
data_sampel1['campaign'].describe()

count    397.000000
mean       2.770781
std        2.445435
min        1.000000
25%        1.000000
50%        2.000000
75%        3.000000
max       16.000000
Name: campaign, dtype: float64

## 2. PENGUJIAN HIPOTESIS

In [ ]:
samples

,index,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,14001,37,management,married,tertiary,no,1403,yes,no,cellular,10,jul,91,15,-1,0,unknown,no
1,32046,36,services,single,secondary,no,-253,yes,no,cellular,14,apr,1073,1,-1,0,unknown,yes
2,13318,47,blue-collar,married,primary,no,-406,no,yes,telephone,8,jul,243,5,-1,0,unknown,no
3,42991,57,housemaid,married,primary,no,501,yes,yes,cellular,11,feb,105,7,188,2,failure,no
4,14237,33,blue-collar,married,secondary,yes,-406,yes,yes,cellular,14,jul,332,2,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,415,34,admin.,divorced,secondary,no,67,yes,no,unknown,6,may,90,1,-1,0,unknown,no
393,43611,45,housemaid,married,secondary,no,360,no,no,cellular,4,may,113,1,182,1,success,no
394,41804,45,housemaid,married,primary,no,8692,yes,no,cellular,14,oct,108,1,-1,0,unknown,no
395,10973,37,management,married,tertiary,no,1170,no,yes,unknown,17,jun,153,3,-1,0,unknown,no


In [ ]:
df1 = samples.groupby(['campaign', 'y']).agg({'index': 'count'}).reset_index()
df1

,campaign,y,index
0,1,no,123
1,1,yes,24
2,2,no,94
3,2,yes,11
4,3,no,46
5,3,yes,6
6,4,no,29
7,4,yes,2
8,5,no,13
9,5,yes,5


In [ ]:
df2 = {'TotalCampaign': [1,2,3,4,5,6,7,8,9,10,11,12,13,15,16],
      'Yes': [24,11,6,2,5,0,0,0,0,0,0,0,0,0,0],
      'No': [123,94,46,29,13,12,7,9,2,5,4,2,1,1,1]}
df2 = pd.DataFrame(df2)
df2

,TotalCampaign,Yes,No
0,1,24,123
1,2,11,94
2,3,6,46
3,4,2,29
4,5,5,13
5,6,0,12
6,7,0,7
7,8,0,9
8,9,0,2
9,10,0,5


In [ ]:
df2['F0_yes'] = df2['Yes']
df2['F0_no'] = df2['No']
df2['Total'] = df2['Yes'] + df2['No']
df2

,TotalCampaign,Yes,No,F0_yes,F0_no,Total
0,1,24,123,24,123,147
1,2,11,94,11,94,105
2,3,6,46,6,46,52
3,4,2,29,2,29,31
4,5,5,13,5,13,18
5,6,0,12,0,12,12
6,7,0,7,0,7,7
7,8,0,9,0,9,9
8,9,0,2,0,2,2
9,10,0,5,0,5,5


In [ ]:
df2['Total'].sum()

397

In [ ]:
df2['FH_Yes'] = (df2['Total']/(df2['Total'].sum()))*(df2['Yes'].sum())
df2['FH_No'] = (df2['Total']/(df2['Total'].sum()))*(df2['No'].sum())
df2

,TotalCampaign,Yes,No,F0_yes,F0_no,Total,FH_Yes,FH_No
0,1,24,123,24,123,147,17.773300,129.226700
1,2,11,94,11,94,105,12.695214,92.304786
2,3,6,46,6,46,52,6.287154,45.712846
3,4,2,29,2,29,31,3.748111,27.251889
4,5,5,13,5,13,18,2.176322,15.823678
5,6,0,12,0,12,12,1.450882,10.549118
6,7,0,7,0,7,7,0.846348,6.153652
7,8,0,9,0,9,9,1.088161,7.911839
8,9,0,2,0,2,2,0.241814,1.758186
9,10,0,5,0,5,5,0.604534,4.395466


In [ ]:
info = [[147,105,52,31,18],[24,11,6,2,5]]
info

[[147, 105, 52, 31, 18], [24, 11, 6, 2, 5]]

In [ ]:
from scipy.stats import chi2_contingency

chi2, p, dof, expected= chi2_contingency(info)
 
alpha = 0.05
print("p value: " + str(p)) 

if p >= alpha:
  print('Tidak cukup bukti menolak H0, maka pengadaan campaign mempengaruhi jumlah pembukaan akun deposit oleh user')
else:
  print('Cukup bukti menolak H0, artinya pengadaan campaign tidak mempengaruhi jumlah pembukaan akun deposit oleh user')


p value: 0.32002278961321534
Tidak cukup bukti menolak H0, maka pengadaan campaign mempengaruhi jumlah pembukaan akun deposit oleh user
